In [29]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
import os
import re
import sys
from joblib import Parallel
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options


In [30]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [31]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [32]:
def get_html_from_url(url):
  chrome_options = Options()
  chrome_options.add_argument("--headless")
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options) 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        flag = 0
  html = browser.page_source
  return html

In [33]:
def convert_time_to_year(time_list):
    today = date.today()
    years_updated = [] 
    updated='UNKNOWN'
    
    for time in time_list:
        time = time.replace(' ago', '')

        time_word = re.sub(r'\d+', '', time)
        time_word.replace(' ', '')

        if 'days' in time_word.lower() or 'hours' in time_word.lower():
            updated = today.year

        if 'months' in time_word.lower():
            time = time.replace(time_word, '')
            try:
                updated = (today - relativedelta(months=int(time))).year
            except:
                print("error in month date")
        
        if 'year' in time_word.lower():
            if 'years' in time_word.lower():
                time_word = time.replace('years','')
            else:
                 time_word = time.replace('year','')
            try:
                updated = today.year - int(time_word)
            except:
                print("error in year date")
        
        years_updated.append(updated)
    return years_updated

In [34]:
def row_index_with_low_upvotes(l):
    to_remove = []
    for i,element in enumerate(l):
        if 'k' in element.lower() or 'm' in element.lower():
            continue
        else:
            if element.lower() == 'vote'or int(element)<500:
                to_remove.append(i)
    return to_remove

In [35]:
url_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\reddit_RCS\first_iteration\sub_reddits_to_scrap.xlsx"
urls = pd.read_excel(url_path)


In [36]:
urls = urls['url'].to_list()

In [37]:
urls

['https://www.reddit.com/r/technology/',
 'https://www.reddit.com/r/tech/',
 'https://www.reddit.com/r/Futurology/',
 'https://www.reddit.com/r/TechnologyPorn/',
 'https://www.reddit.com/r/hardware/',
 'https://www.reddit.com/r/science/',
 'https://www.reddit.com/r/CryptoCurrency/',
 'https://www.reddit.com/r/Bitcoin/']

In [38]:
cols = ['domains', 'posts', 'upvotes', 'last_updated', 'discussion_links']
df_all = pd.DataFrame([], columns=cols)


for url in urls:
    page_html = get_html_from_url(url)
    #go to filter by flair and extract domain pages
    soup = BeautifulSoup(page_html)
    big_box = soup.find("div", {"class":"rpBJOHq2PR60pnwJlUyP0"})
    boxes = soup.find_all("div", {"data-testid":"post-container"})
    posts = [box.find('h3', {"class":"_eYtD2XCVieq6emjKBH3m"}).find_next(text=True).strip() for box in boxes]
    upvotes  = [box.find('div', {"class":"_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ"}).find_next(text=True).strip() for box in boxes]

    domains = []
    
    for box in boxes:
        domain = box.find('span', {
            'class':"_1jNPl3YUk6zbpLWdjaJT1r _2VqfzH0dZ9dIl3XWNxs42y aJrgrewN9C8x1Fusdx4hh _1Dl-kvSxyJMWO9nuoTof8N"
            })
        if domain is None:
            domains.append(np.nan)
    
        else:
            domains.append(domain.find_next(text=True).strip())
    
    time_updated = [box.find('span', {'class':"_2VF2J19pUIMSLJFky-7PEI"}).find_next(text=True).strip() for box in boxes]
    time_updated = convert_time_to_year(time_updated)

    links_to_discussions = [] 
    for i, box in enumerate(boxes):
        add_link = 'UNKNOWN'
        try:
           add_link = box.find('a', {'class':"_13svhQIUZqD9PVzFcLwOKT styled-outbound-link"})['href']
        except:
           print('No link found in box', i)
        links_to_discussions.append(add_link)
    
    df = pd.DataFrame([])
    df['domains'] = pd.DataFrame(domains)
    df['posts'] = pd.DataFrame(posts)
    df['upvotes'] = pd.DataFrame(upvotes)
    df['last_updated'] = pd.DataFrame(time_updated)
    df['discussion_links'] = pd.DataFrame(links_to_discussions)

    #remove posts with very low upvotes
    idx_low_upvotes =row_index_with_low_upvotes(upvotes)
    df.drop(idx_low_upvotes, axis=0, inplace=True)
    print(len(df))
    df_all = pd.concat([df_all, df], axis=0)


C:\Users\Ravit\AppData\Local\Temp\ipykernel_17308\3039072896.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options)


No link found in box 0
134


C:\Users\Ravit\AppData\Local\Temp\ipykernel_17308\3039072896.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options)


No link found in box 374
No link found in box 393
No link found in box 400
No link found in box 425
No link found in box 427
124


C:\Users\Ravit\AppData\Local\Temp\ipykernel_17308\3039072896.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options)


...

No link found in box 0
No link found in box 1
No link found in box 29
No link found in box 58
No link found in box 63
No link found in box 92
No link found in box 93
No link found in box 101
No link found in box 103
No link found in box 123
No link found in box 124
No link found in box 141
No link found in box 145
No link found in box 158
No link found in box 171
No link found in box 174
No link found in box 176
No link found in box 181
No link found in box 182
No link found in box 187
No link found in box 201
No link found in box 206
No link found in box 207
No link found in box 215
No link found in box 221
No link found in box 224
No link found in box 239
No link found in box 243
No link found in box 250
No link found in box 251
No link found in box 262
No link found in box 271
No link found in box 279
No link found in box 304
No link found in box 309
No link found in box 311
No link found in box 314
No link found in box 318
No link found in box 324
No link found in box 325
No link f

C:\Users\Ravit\AppData\Local\Temp\ipykernel_17308\3039072896.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options)


No link found in box 693
240


C:\Users\Ravit\AppData\Local\Temp\ipykernel_17308\3039072896.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options)


TimeoutException: Message: timeout: Timed out receiving message from renderer: 28.832
  (Session info: headless chrome=107.0.5304.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C6ACD3+2075859]
	Ordinal0 [0x00BFEE61+1633889]
	Ordinal0 [0x00AFB7BD+571325]
	Ordinal0 [0x00AEE410+517136]
	Ordinal0 [0x00AEE231+516657]
	Ordinal0 [0x00AED0A3+512163]
	Ordinal0 [0x00AED6D8+513752]
	Ordinal0 [0x00AF6662+550498]
	Ordinal0 [0x00B01BD5+596949]
	Ordinal0 [0x00B04FC0+610240]
	Ordinal0 [0x00AEDA81+514689]
	Ordinal0 [0x00B01A0F+596495]
	Ordinal0 [0x00B5B93B+964923]
	Ordinal0 [0x00B47136+880950]
	Ordinal0 [0x00B1FEFD+720637]
	Ordinal0 [0x00B20F3F+724799]
	GetHandleVerifier [0x00F1EED2+2769538]
	GetHandleVerifier [0x00F10D95+2711877]
	GetHandleVerifier [0x00CFA03A+521194]
	GetHandleVerifier [0x00CF8DA0+516432]
	Ordinal0 [0x00C0682C+1665068]
	Ordinal0 [0x00C0B128+1683752]
	Ordinal0 [0x00C0B215+1683989]
	Ordinal0 [0x00C16484+1729668]
	BaseThreadInitThunk [0x772DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77C2

In [39]:
df_all

,domains,posts,upvotes,last_updated,discussion_links
1,Social Media,FCC commissioner calls for TikTok ban,1.9k,2022,https://www.cnn.com/2022/11/02/tech/fcc-commis...
2,Business,Adobe Photoshop Designers Are Furious That Pan...,773,2022,https://gizmodo.com/pantone-adobe-photoshop-15...
3,Business,Binance CEO says he anticipates 90% of Elon Mu...,34.3k,2022,https://www.businessinsider.com/binance-ceo-sa...
4,Business,Uber whistleblower says current business model...,2.7k,2022,https://www.reuters.com/technology/uber-whistl...
5,Social Media,Twitter will not reinstate banned users withou...,4.0k,2022,https://www.reuters.com/technology/twitter-wil...
...,...,...,...,...,...
694,Economics,More than a quarter of vacuum cleaners sold on...,41.0k,2022,https://www.futurity.org/amazon-discounts-cons...
698,Health,Heavy-load exercise in older adults activates ...,1.5k,2022,https://eurapa.biomedcentral.com/articles/10.1...
701,Health,Babies have air pollution in their lungs and b...,49.4k,2022,https://www.abdn.ac.uk/news/16424/
704,Biology,New study shows IVF favours different traits i...,2.8k,2022,https://academic.oup.com/humupd/article/28/4/4...


In [ ]:
df_all = pd.DataFrame(df_all)

In [ ]:
save_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\reddit_RCS\first_iteration\scrapped_posts_first_iteartion.csv"
df_all.to_csv(save_path)